# 모듈 설치

In [ ]:
!pip install yfinance

In [ ]:
!pip install ta

In [ ]:
!pip install --upgrade mplfinance

In [ ]:
!pip install https://github.com/matplotlib/mpl_finance/archive/master.zip

In [ ]:
!pip install statsmodels

In [ ]:
!pip install tensorflow

In [ ]:
!pip install shap

In [ ]:
!pip install pdpbox

# 데이터 분석 시작

In [ ]:
import warnings
import os
import ta
import datetime
import yfinance as yf
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_finance import candlestick_ohlc
import statsmodels.api as sm
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error, mean_squared_error,  r2_score, mean_absolute_error
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Dense, SimpleRNN
from tensorflow.keras.layers import LSTM
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from xgboost import XGBRegressor
import sklearn.metrics as metrics
from sklearn.linear_model import LinearRegression 
from sklearn.neural_network import MLPRegressor 
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.svm import SVR 
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import cross_val_predict 
from sklearn.metrics import r2_score
from sklearn.model_selection import TimeSeriesSplit
from scipy.stats import randint, uniform
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
import shap
from pdpbox.pdp import pdp_isolate, pdp_plot
from pdpbox.pdp import pdp_interact, pdp_interact_plot

In [ ]:
apple = yf.download('AAPL',start = '1989-10-06')

In [ ]:
apple1 = apple.copy()
apple1.columns = ['Open', 'High', 'Low', 'Close', 'Adj', 'Vol']

apple1

In [ ]:
# 수정종가가 아닌 종가에 대한 이동평균선, RSI, 볼린저 상한밴드, 볼리저 하한밴드, MACD 생성

ma12 = apple1['Close'].rolling(window = 12).mean()
ma26 = apple1['Close'].rolling(window = 26).mean()
RSI = ta.momentum.rsi(apple1['Close'])
BOL_H = ta.volatility.bollinger_hband(apple1['Close'])
BOL_L = ta.volatility.bollinger_lband(apple1['Close'])
MACD = ta.trend.macd(apple1['Close'])

In [ ]:
# RSI, 볼린저 상한밴드, 볼린저 하한밴드, MACD의 항목을 추가 

apple2 = pd.concat([apple1, ma12, ma26, RSI, BOL_H, BOL_L, MACD], axis = 1)

apple2.columns = ['Open', 'High', 'Low', 'Close', 'Adj', 'Vol', 'MA_12', 'MA_26', 'RSI', 'BOL_H', 'BOL_L', 'MACD']

apple2

In [ ]:
signal = apple2['MACD'].ewm(9).mean()

apple3 = pd.concat([apple2, signal], axis = 1)

apple3.columns = ['Open', 'High', 'Low', 'Close', 'Adj', 'Vol', 'MA_12', 'MA_26', 'RSI', 'BOL_H', 'BOL_L', 'MACD', 'SIGNAL']

apple3

In [ ]:
apple3.isnull().sum() # 결측치 확인

In [ ]:
# 결측치 수가 굉장히 미미한 수준이기 때문에 제거함

apple3 = apple3.dropna()

apple3

In [ ]:
apple3.dtypes

# 각 Column 의미

1. Date : 날짜

2. Open : 장 시작시 애플의 가격

3. High : 장 중 애플의 최고 가격

4. Low : 장 중 애플의 최저 가격

5. Close : 장 마감시 애플의 가격

6. Adj : 장 후 애플의 수정 종가

7. Vol : 애플 거래량

8. MA_12 : 12일 이동평균선

9. MA_26 : 26일 이동평균선

10. RSI : 상대 강도 지수

11. BOL_H : 볼린저 상한밴드

12. BOL_L : 볼린저 하한밴드

13. MACD : 추세 분석 지표

14. SIGNAL : MACD 상대 반응모델 선


*   수정 종가 = 분할, 배당, 배분, 신주 발생이 된 경우를 고려해 주식 가격을 조정(adjustment)해둔 가격. 따라서 굉장히 중요함.

*   이동평균선 = 일정 기간동안 주가를 산술평균낸 후 값을 연결한 선으로 5일선은 대표적 단기 이동평균선, 60일선은 대표적 장기 이동평균선이다.

*   상대 강도 지수 = 과매수 상태인가, 과매도 상태인가를 나타내는 지표.

*   볼린저밴드 = 이동평균선을 중심으로 하는 표준편차. 위쪽으로 상한밴드, 아래쪽으로 하한밴드.

*   추세 분석 지표 = 단기 이동평균선 - 장기 이동평균선



In [ ]:
# 세부 지표 파악

apple3.describe()

In [ ]:
# 타겟 세부 지표 파악

target1 = apple3['High']
target2 = apple3['Low']

target1.describe(), target2.describe()

In [ ]:
# 각 column 들의 분포

apple3.hist(bins = 20, grid = True, figsize = (16, 12))

plt.show()

In [ ]:
# 피어슨 상관 계수 분석

corr = apple3.corr(method = 'pearson')

corr

In [ ]:
#변수 끼리의 상관관계 분석

apple3.corr().loc[apple3.columns, apple3.columns].style.background_gradient().set_precision(2).set_properties(**{'font-size': '11pt'})

1. APPLE의 지표들에 대한 피어슨 상관 계수를 나타내었을 때, 거래량만 음의 상관관계를 나타내지만 큰 수치가 아닌 상관관계이다

2. MACD 역시 큰 상관관계라고는 볼 수 가 없다.

In [ ]:
# 최근 60거래일 거래현황 시각화

apple60 = apple3[-120:]

In [ ]:
# 60 거래일에 관한 거래현황

fig = plt.figure(figsize = (16, 10))
fig.set_facecolor('lightgrey')
gs = gridspec.GridSpec(4, 1, height_ratios=[5, 1, 1, 1])
axes = []
axes.append(plt.subplot(gs[0]))
axes.append(plt.subplot(gs[1], sharex = axes[0]))
axes.append(plt.subplot(gs[2], sharex = axes[1]))
axes.append(plt.subplot(gs[3], sharex = axes[2]))
axes[0].get_xaxis().set_visible(False)

x = np.arange(len(apple60.index))
ohlc = apple60[['Open', 'High', 'Low', 'Close']].astype(int).values
dohlc = np.hstack((np.reshape(x, (-1, 1)), ohlc))

candlestick_ohlc(axes[0], dohlc, width = 0.5, colorup = 'r', colordown = 'b')
axes[1].bar(x, apple60.Vol, color = 'black', width = 0.6, align = 'center')
axes[2].plot(x, apple60.RSI, color = 'red')
axes[3].plot(x, apple60.MACD, color = 'green')
axes[3].plot(x, apple60.SIGNAL, color = 'red')
axes[3].axhline(0, color = 'black')
plt.tight_layout()

plt.show();

# VIF (Variance Infliation Factor)
독립변수를 다른 독립변수들로 선형회귀한 성능을 나타낸다

의존성이 낮은(분산이 작은) 독립변수를 선택하거나, 의존성이 높은(분산이 높은) 독립변수를 제거하며 사용한다


*    variance_inflation_factor(X, i) : Xi를 x나머지로 회귀분석한 후 VIF값을 구한것. 즉 xi의 vif값. 즉 이값이 높을수록 종속성이 높다는 뜻

In [ ]:
vif = pd.DataFrame()

vif['VIF_Factor'] = [variance_inflation_factor(apple3.values, i) 
                     for i in range(apple3.shape[1])]
vif['Feature'] = apple3.columns
vif.sort_values(by = 'VIF_Factor', ascending = True)

시계열 데이터이기 때문에 과적합이 발생할 수 있지만 이 flow 를 타고 예측을 한다는 전제하에 그대로 진행을 해보자


# 안정성(정상성) 확인


In [ ]:
# 시계열이 정상성이있는지 귀무가설로 판단해보자

def adf_test(df):
  result = adfuller(df.values)
  print('ADF Statics: %f' % result[0])
  print('P-value: %f' % result[1])
  print('Critical Values:')
  for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))

adf_test(apple3['High'])

p-value가 0.05보다 높기때문에 시계열은 안정적이지 않다는 귀무가설을 기각할 수 없다.

시계열이 안정적이지 않다는 것

In [ ]:
def kpss_test(df):
  statistic, p_value, n_lags, critical_values = kpss(df.values)

  print(f'KPSS Statics: {statistic}')
  print(f'P-value: {p_value}')
  print(f'Num-lags: {n_lags}')
  print('Critical Values:')
  for key, value in critical_values.items():
    print(f' {key} : {value}')

kpss_test(apple3['High'])

p-value가 0.05보다 낮기 때문에 시계열은 안정적이라는 귀무가설을 기각한다

결국 시계열은 역시 안정적이지 않다는 것이다.

# 안정적인 모습의 데이터일까 

주가데이터라 그런지 비슷한 형태의 wave를 가지고 일정 주기동안 반복하는 형태인 정상성을 분석한 결과 결국 데이터가 안정적인 모습이 아니라고 판단된다.

In [ ]:
# Train, Test 셋을 나눈다 - 코로나 범유행 기간을 기준으로 한다

def datasplit_ts(df, y_colname, X_colname, criteria):
    train = df.loc[df.index < criteria,:]
    test = df.loc[df.index >= criteria,:]
    y_train = train[y_colname]
    X_train = train[X_colname]
    y_test = test[y_colname]
    X_test = test[X_colname]
    print('Train_size:', train.shape, 'Test_size:', test.shape)
    print('X_train:', X_train.shape, 'y_train:', y_train.shape)
    print('X_test:', X_test.shape, 'y_test:', y_test.shape)
    return X_train, X_test, y_train, y_test

In [ ]:
# target을 High일 때와 Low일 때 둘로 나눈다

y_colname1 = ['High']
y_colname2 = ['Low']
X_remove1 = ['Date', 'Low']
X_remove2 = ['Date', 'High']
X_colname1 = [x for x in apple3.columns if x not in y_colname1 + X_remove1]
X_colname2 = [x for x in apple3.columns if x not in y_colname2 + X_remove2]
X_train1, X_test1, y_train1, y_test1 = datasplit_ts(apple3, y_colname1, X_colname1, '2020-01-02')
X_train2, X_test2, y_train2, y_test2 = datasplit_ts(apple3, y_colname2, X_colname2, '2020-01-02')

In [ ]:
# Scaling 진행

def feature_engineering_scaling(scaler, X_train, X_test):
    # scaler파라미터는 아래 4개중 하나를 넣는다
    # preprocessing.MinMaxScaler()
    # preprocessing.StandardScaler()
    # preprocessing.RobustScaler()
    # preprocessing.Normalizer()
    scaler = scaler
    scaler_fit = scaler.fit(X_train)
    X_train_scaling = pd.DataFrame(scaler_fit.transform(X_train), 
                               index=X_train.index, columns=X_train.columns)
    X_test_scaling = pd.DataFrame(scaler_fit.transform(X_test), 
                               index=X_test.index, columns=X_test.columns)
    return X_train_scaling, X_test_scaling

In [ ]:
X_train_sc1, X_test_sc1 = feature_engineering_scaling(preprocessing.MinMaxScaler(), X_train1, X_test1)
X_train_sc2, X_test_sc2 = feature_engineering_scaling(preprocessing.MinMaxScaler(), X_train2, X_test2)

In [ ]:
predict1 = apple3['High'].mean()
predict2 = apple3['Low'].mean()

predict1, predict2

In [ ]:
# 기준모델에 대한 시각화

x1 = apple3.index
y1 = apple3['High']

errors = predict1 - apple['High']
mean_absolute_error = errors.abs().mean()

sns.lineplot(x = x1, y = predict1, color = 'red')
sns.scatterplot(x = x1, y = y1, color = 'blue');

In [ ]:
x2 = apple3.index
y2 = apple3['Low']

errors = predict1 - apple['Low']
mean_absolute_error = errors.abs().mean()

sns.lineplot(x = x2, y = predict2, color = 'red')
sns.scatterplot(x = x2, y = y2, color = 'blue');

# 모델링 시작

1. OLS 모델

2. Ridge

3. RandomForest를 통한 배깅

4. LGBM을 통한 부스팅

In [ ]:
def evaluation(y_real, y_pred, graph_on = False):
    loss_length = len(y_real.values.flatten()) - len(y_pred)
    if loss_length != 0:
        y_real = y_real[loss_length:]
    if graph_on == True:
        pd.concat([y_real, pd.DataFrame(y_pred, index = y_real.index, columns = ['prediction'])], axis = 1).plot(kind = 'line', figsize = (20,6),
                                                                                                           xlim=(y_real.index.min(),y_real.index.max()),
                                                                                                           linewidth = 3, fontsize = 20)
        plt.title('Time Series of Target', fontsize = 20)
        plt.xlabel('Index', fontsize = 15)
        plt.ylabel('Target Value', fontsize = 15)
    MAE = abs(y_real.values.flatten() - y_pred).mean()
    MSE = ((y_real.values.flatten() - y_pred)**2).mean()
    MAPE = (abs(y_real.values.flatten() - y_pred)/y_real.values.flatten()*100).mean()
    Score = pd.DataFrame([MAE, MSE, MAPE], index=['MAE', 'MSE', 'MAPE'], columns = ['Score']).T
    Residual = pd.DataFrame(y_real.values.flatten() - y_pred, index = y_real.index, columns = ['Error'])
    return Score, Residual

In [ ]:
def evaluation_trte(y_real_train, y_pred_train, y_real_test, y_pred_test, graph_on = False):
    Score_tr, Residual_tr = evaluation(y_real_train, y_pred_train, graph_on = graph_on)
    Score_te, Residual_te = evaluation(y_real_test, y_pred_test, graph_on = graph_on)
    Score_trte = pd.concat([Score_tr, Score_te], axis=0)
    Score_trte.index = ['Train', 'Test']
    return Score_trte, Residual_tr, Residual_te

In [ ]:
fit_ols1 = sm.OLS(y_train1, X_train1).fit()
display(fit_ols1.summary())

In [ ]:
fit_ols2 = sm.OLS(y_train1, X_train_sc1).fit()
display(fit_ols2.summary()) 

In [ ]:
# OLS 기법 기준모델 설정

pred_train_ols1 = fit_ols1.predict(X_train1).values
pred_test_ols1 = fit_ols1.predict(X_test1).values

# 예측값 평가

Score_reg1, Resid_train_reg1, Resid_test_reg1 = evaluation_trte(y_train1, pred_train_ols1, y_test1, pred_test_ols1, graph_on = True)
print(Score_reg1)
print(Resid_train_reg1)
print(Resid_test_reg1)

In [ ]:
# 스케일링된 자료를 통한 설정

pred_train_ols2 = fit_ols2.predict(X_train_sc1).values
pred_test_ols2 = fit_ols2.predict(X_test_sc1).values

# 예측값 평가

Score_reg2, Resid_train_reg2, Resid_test_reg2 = evaluation_trte(y_train1, pred_train_ols2, y_test1, pred_test_ols2, graph_on = True)
print(Score_reg2)
print(Resid_train_reg2)
print(Resid_test_reg2)

In [ ]:
alphas = [0, 0.1, 1, 10, 100, 1000, 10000, 100000]

# alpha list 값을 반복하면서 alpha 에 따른 평균 r제곱을 구함

for alpha in alphas:

	# alpha 값들이 for문을 돌면서 새로운 ridge 회귀 객체를 생성

    ridge = Ridge(alpha = alpha)
    
    # cross_val_score를 이용해 5 폴드의 평균 RMSE를 계산
    
    model1 = Ridge(alpha = alpha, fit_intercept = True, normalize = True, random_state = 2).fit(X_train1, y_train1)
    model1.predict(X_train1)
    y_pred1 = model1.predict(X_test1)
    
    # alpha 값의 가정에 따라 평균 5 folds의 R제곱이 어떻게 계산되는지 출력

    print('alpha {0}일때 5 folds의 평균 Score: {1:.3f}'.format(alpha, model1.score(X_test1, y_test1)))

결국 alpha 값은 1일 때 가장 안정적인 모델이라고 보인다. 0.1 이하일 때는 과적합의 모습이 나타나고 10부터는 폐기 모델이 된다.

In [ ]:
model1 = Ridge(alpha = 1, fit_intercept = True, normalize = True, random_state = 2).fit(X_train1, y_train1)

model1.predict(X_train1)
y_pred1 = model1.predict(X_test1)

print(f'훈련 모델 Score :{model1.score(X_train1, y_train1) : .3f}')
print(f'Test 모델 Score :{model1.score(X_test1, y_test1) : .3f}')
print(f'Test 모델 MSE :{mean_squared_error(y_test1, y_pred1) : .3f}')

In [ ]:
model1.predict(X_train_sc1)
y_sc_pred1 = model1.predict(X_test_sc1)

print(f'훈련 모델 Score :{model1.score(X_train_sc1, y_train1) : .3f}')
print(f'Test 모델 Score :{model1.score(X_test_sc1, y_test1) : .3f}')
print(f'Test 모델 MSE :{mean_squared_error(y_test1, y_sc_pred1) : .3f}')

스케일링을 안한 데이터와 진행한 데이터를 비교한 결과 score에서 나타나듯 스케일링을 안한 데이터가 더 좋다고 보여진다.

In [ ]:
# Cross_Val_Score 로 교차검증 일반화 성능 검사

cross_val_score(model1, X_train1, y_train1, scoring = None, cv = None)

In [ ]:
pipe = make_pipeline(
    RandomForestRegressor(random_state = 2)
)

dists = {
    'randomforestregressor__n_estimators': randint(10, 100, 500), 
    'randomforestregressor__max_depth': [5, 10, 20, 40, None], 
    'randomforestregressor__min_samples_leaf' : [8, 16, 32, None],
    'randomforestregressor__min_samples_split' : [8, 16, 32, None]
}

clf = RandomizedSearchCV(
    pipe, 
    param_distributions = dists, 
    n_iter = 10, 
    cv = 3, 
    scoring = 'neg_mean_absolute_error',  
    verbose = 1,
    n_jobs = -1
)
clf.fit(X_train1, y_train1);

In [ ]:
print('최적 하이퍼파라미터: ', clf.best_params_)
print('MAE: ', -clf.best_score_)

In [ ]:
model2 = RandomForestRegressor(random_state = 2, max_depth = None, min_samples_leaf = 16, min_samples_split = 8, n_estimators = 549)

model2.fit(X_train1, y_train1)
y_pred2 = model2.predict(X_test1)

print(f'훈련 모델 Score :{model2.score(X_train1, y_train1) : .3f}')
print(f'Test 모델 Score :{model2.score(X_test1, y_test1) : .3f}')
print(f'Test 모델 MSE :{mean_squared_error(y_test1, y_pred2) : .3f}')

In [ ]:
clf.fit(X_train_sc1, y_train1);

In [ ]:
print('최적 하이퍼파라미터: ', clf.best_params_)
print('MAE: ', -clf.best_score_)

In [ ]:
model2 = RandomForestRegressor(random_state = 2, max_depth = 10, min_samples_leaf = 16, min_samples_split = 8, n_estimators = 594)

model2.fit(X_train_sc1, y_train1)
y_sc_pred2 = model2.predict(X_test_sc1)

print(f'훈련 모델 Score :{model2.score(X_train_sc1, y_train1) : .3f}')
print(f'Test 모델 Score :{model2.score(X_test_sc1, y_test1) : .3f}')
print(f'Test 모델 MSE :{mean_squared_error(y_test1, y_sc_pred2) : .3f}')

In [ ]:
pipe1 = make_pipeline(
    LGBMRegressor(random_state = 2)
)

dists1 = {
    'lgbmregressor__n_estimators': randint(10, 100, 500), 
    'lgbmregressor__learning_rate': [0.05, 0.1, 0.2, 0.40]
}

clf1 = RandomizedSearchCV(
    pipe1, 
    param_distributions = dists1, 
    n_iter = 10, 
    cv = 3, 
    scoring = 'neg_mean_absolute_error',  
    verbose = 1,
    n_jobs = -1
)
clf1.fit(X_train1, y_train1);

In [ ]:
print('최적 하이퍼파라미터: ', clf1.best_params_)
print('MAE: ', -clf1.best_score_)

In [ ]:
model3 = LGBMRegressor(learning_rate = 0.05, n_estimators = 550, random_state = 2).fit(X_train1, y_train1)

model3.predict(X_train1)
y_pred3 = model3.predict(X_test1)

print(f'훈련 모델 Score :{model3.score(X_train1, y_train1) : .3f}')
print(f'Test 모델 Score :{model3.score(X_test1, y_test1) : .3f}')
print(f'Test 모델 MSE :{mean_squared_error(y_test1, y_pred3) : .3f}')

# 모델 별 평가 결과

1. model1(Ridge회귀 사용)은 1에 가까운 평가 지표가 나왔다. 이는 아마 과적합이 의심된다. alpha 값을 조정해서 수치를 변경해준다.

2. model2(RandomForestRegressor)와 model3(LGBMRegressor)의 경우 test셋에서 음수의 r제곱 값의 평가 지표가 나왔다. 폐기를 해야할 정도로 안좋은 모델이라고 생각한다.

3. 다른 target으로 잡은 Low 도 같은 방법으로 사용할 수 있다.

In [ ]:
apple_tomorrow = pd.DataFrame({'Open' : [143], 'Close' : [146], 'Adj' : [146] , 'Vol' : [109742900], 'NA_12' : [150], 'NA_26' : [158], 'RSI' : [45] , 'BOL_H' : [170], 'BOL_L' : [142], 'MACD' : [-5.21], 'SIGNAL' : [-3.03]})

apple_tomorrow

In [ ]:
model1 = Ridge(alpha = 1, fit_intercept = True, normalize = True, random_state = 2).fit(X_train1, y_train1)

X_train1.columns

In [ ]:
%matplotlib inline
import seaborn as sns
sns.displot(y_train1, kde = True);

In [ ]:
print('R^2', model1.score(X_test1, y_test1))

In [ ]:
isolated = pdp_isolate(
    model = model1, 
    dataset = X_test1, 
    model_features = X_test1.columns, 
    feature = 'Open',
    grid_type = 'percentile',
    num_grid_points = 10
)

pdp_plot(isolated, feature_name = 'Open', plot_lines = True)

In [ ]:
isolated = pdp_isolate(
    model = model1, 
    dataset = X_test1, 
    model_features = X_test1.columns, 
    feature = 'Vol',
    grid_type = 'percentile',
    num_grid_points = 10
)

pdp_plot(isolated, feature_name = 'Vol', plot_lines = True)

In [ ]:
isolated = pdp_isolate(
    model = model1, 
    dataset = X_test1, 
    model_features = X_test1.columns, 
    feature = 'RSI',
    grid_type = 'percentile',
    num_grid_points = 10
)

pdp_plot(isolated, feature_name = 'RSI', plot_lines = True)

In [ ]:
isolated = pdp_isolate(
    model = model1, 
    dataset = X_test1, 
    model_features = X_test1.columns, 
    feature = 'MACD',
    grid_type = 'percentile',
    num_grid_points = 10
)

pdp_plot(isolated, feature_name = 'MACD', plot_lines = True)

# PDP를 사용해보자

1. 일단 연관성이 그렇게 짙게 나오지 않는 column 들을 가지고 PDP분석을 해보자

2. 시초가, RSI, MACD의 경우는 양의 선형관계로 타겟 변수에 같은 방향의 영향을 미친다고 나타난다.

3. 거래량의 경우는 음의 선형관계로 타겟 변수와 반대 방향을 주는 영향성을 나타낸다고 알 수 있다.

In [ ]:
model4 = Ridge(alpha = 0.5, fit_intercept = True, normalize = True, random_state = 2).fit(X_train1, y_train1)

model4.predict(X_train1)
y_pred4 = model4.predict(X_test1)

print(f'훈련 모델 Score :{model4.score(X_train1, y_train1) : .3f}')
print(f'Test 모델 Score :{model4.score(X_test1, y_test1) : .3f}')
print(f'Test 모델 MSE :{mean_squared_error(y_test1, y_pred4) : .3f}')

In [ ]:
tomo = model4.predict(apple_tomorrow)

tomo

1. 오차 범위가 크게 나오기 때문에 이정도의 예측치라면 나쁘지는 않을지 모른다

2. RMSE 값이 약 8 정도 나온다.

3. 최고 값을 구해야 하기 때문에 나온 값과 RMSE 값을 더해 줘 보자

4. 144~145 정도의 값이 나온다.

5. 오늘 시작하는 장에서 136 ~ 145 사이의 값이 나올 것이라고 예측이 가능하다.